<a href="https://colab.research.google.com/github/ThaoPham96/DeepARV/blob/main/DeepARV_Sim_construction_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**DeepARV: Ensemble Deep Learning to Predict Drug-Drug Interaction (DDI) of Clinical Relevance with Antiretrovial Therapy (ARV)**

In this study, we built an esemble learning algorithm to predict DDIs of clinical relevance between ARVs and comedications.

DDI severity grading between 29,142 drug pairs was extracted from the Liverpool HIV Drug Interaction database, https://www.hiv-druginteractions.org/. Molecular structure of each drug was converted to Morgan fingerprints where each atom was represented in a numerical format and compared by constructing drug similarity profiles. We developed DeepARV, a feed-forward neural network where drug similarity profiles were fed into the input layer and outputted 4 categories of DDI as 'traffic light' system:

*   Red: drugs should not be co-administered
*   Amber: interaction of potential clinical relevance manageable by monitoring/dose adjustment
*   Yellow: interaction of weak relevance
*   Green: no expected interaction

The imbalance in the distribution of DDI severity grades was addressed by undersampling and applying ensemble learning during training of DeepARV.

# Imports and installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorboard
!pip install tensorflow_addons

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import keras
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.callbacks import Callback
import tensorflow.experimental.numpy as tnp
import math
import timeit
import datetime, os
import pandas as pd
from numpy import argmax
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from sklearn.utils import class_weight
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm import tqdm
from deep_arv_utils import *
from matplotlib.colors import LinearSegmentedColormap
from tensorflow.keras.callbacks import EarlyStopping

# DeepARV Class

In [ ]:
class DeepARV_ensemble:
  ''' Training of DeepARV '''
  def __init__(self, train_df=None, test_df=None, n_models=5):

    # Load data
    self.train_df = train_df
    self.test_ds = test_df
    self.n_models = n_models


  def build_model(self, feature):
    ''' Builds a single model '''
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_shape=(feature,))),
    model.add(Dropout(0.2)),
    model.add(Dense(512, activation='relu')),
    model.add(Dropout(0.2)),
    model.add(Dense(256, activation='relu')),
    model.add(Dropout(0.2)),
    model.add(Dense(128, activation='relu')),
    model.add(Dropout(0.2)),
    model.add(Dense(4, activation='softmax'))
    tf.keras.optimizers.Adam(learning_rate=0.001,
                            beta_1=0.9, beta_2=0.999,
                            epsilon=1e-07,
                            amsgrad=False,
                            name='Adam')
    model.compile(optimizer='Adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    self.model = model
    return model

  #Undersample the majority
  def undersample(self ):
      minority = self.train_df[self.train_df.Label != 0]
      majority = self.train_df[self.train_df.Label == 0
                               ].sample(frac=1, random_state=42
                                        ).reset_index(drop=True)

      # Split majority class for undersampling
      majority_dict = split_majority(majority,5,2762)
      train_split_dict = {} #dict to hold final training df
      n_model = 5
      self.undersampled_feature_dict = {}
      self.undersampled_label_dict = {}
      print('\n########################################\n')
      print('---------------------------------------')
      print('DDI 4-fold training distribution BEFORE undersampling : ',
            count_uniques_ddi(self.train_df))
      print('---------------------------------------')

      for i in range(5):
          train_split_dict[i] = pd.concat([majority_dict[i],minority]
                                          ).sample(frac=1,random_state=42
                                          ).reset_index(drop=True)
          print('---------------------------------------')
          print('DDI distribution for training model '+str(i)+
                ' AFTER undersampling: \n',
                count_uniques_ddi(train_split_dict[i]))
          self.undersampled_feature_dict[i] = train_split_dict[i
                                                               ].drop(
                                                              columns='Label')
          self.undersampled_label_dict[i] = train_split_dict[i].Label.dropna()
      return

  def train(self, n_models=5):
        print('\n########################################\n')
        print('Building models, balancing with class weights and training ...\n')
        self.model_dict = {}
        for i in range(n_models):
            print('Training model: ', i)
            class_weights = class_weight.compute_class_weight(
                            class_weight='balanced',
                            classes = np.unique(self.undersampled_label_dict[i]),
                            y = self.undersampled_label_dict[i])
            class_weights_model = dict(zip(np.unique(
                                  self.undersampled_label_dict[i]),
                                           class_weights))
            self.model_dict[i] = self.build_model(1376)
            early_stopping = EarlyStopping(monitor='val_loss',
                                           patience=10,
                                           restore_best_weights=True)
            self.model_dict[i].fit(self.undersampled_feature_dict[i],
                                   self.undersampled_label_dict[i],
                                   epochs=30,
                                   validation_split=0.1,
                                   batch_size = 128,
                                   class_weight=class_weights_model,
                                   callbacks=[early_stopping])

  def test(self):
      # Predict from all models
      print('\nPredicting for test set for each model, aggregating for final vote...\n')
      all_results = {}
      for i in range(5):
          all_results[i] = self.model_dict[i].predict(self.test_ds.iloc[:,:-1])

      all_results_values = np.array(list(all_results.values()))
      soft_vote = all_results_values.mean(axis=0).argmax(axis=1)

      self.df_vote = pd.DataFrame(data=soft_vote, columns=['final_vote'])

      self.macro_results, self.each_class_results = calculate_macro_metrics(
                                                    self.df_vote , self.test_ds)


      self.all_results_df = pd.DataFrame(self.each_class_results,
                                          index=['Green','Yellow',
                                                'Amber','Red','Macro'],
                            columns =['Accuracy','Precision','Sensitivity',
                                'Specificity','F1_score', 'Balanced Accuracy'])

  def save_models(self,ensemble_id):
    path_to_save = '/content/drive/MyDrive/liverpool_hiv/DeepARV_revised/v2_revised/' + str(ensemble_id)
    os.makedirs(path_to_save, exist_ok=True)
    for i in range(self.n_models):
      print('Saving iteration...')
      self.model_dict[i].save(path_to_save + '/DeepARV_model'+ str(i)+'.keras')
    print('Cross Validation Models ',  str(ensemble_id) ,' saved.')

  def visualize_confusion_matrix(self):
    print('\n ------------------------------')
    print('Plotting confusion matrix ...')
    y_pred = self.df_vote.final_vote
    y_test = self.test_ds['label']
    plot_norm_confusion_matrix(y_pred, y_test)
    plot_confusion_matrix(y_pred, y_test)





# Training - 5 fold cross validation

In [ ]:
def DeepARV_5_fold_cross_validation():

  # Load training data into dictionary with 5 folds
  train_5fold = load_training_data()

  #Store accuracy, precision,  sensitivity, specificity, f1_score, BAcc for each
  # class and macro.
  cv5_macro_results = np.zeros((5,6))
  cv5_each_class_results = []

  # Storage for each ensemble
  ensemble_dict = {}
  # iterate through each fold as test, remaining as training
  for f in range(5):
    print('\n########################################')
    print('\nRunning fold: ', f)
    folds = [i for i in range(5)]
    test_idx = folds.pop(f)
    test_ds = train_5fold[test_idx]
    train_ds = pd.concat([train_5fold[folds[0]], train_5fold[folds[1]],
                  train_5fold[folds[2]], train_5fold[folds[3]]])

    # Build ensemble, undersample to balance data, train and test.
    deepARV_ensemble = DeepARV_ensemble(train_ds, test_ds)
    deepARV_ensemble.undersample()
    deepARV_ensemble.train()
    deepARV_ensemble.test()
    deepARV_ensemble.save_models(f)
    ensemble_dict[f] = deepARV_ensemble.model_dict
    cv5_macro_results[f] = deepARV_ensemble.macro_results
    cv5_each_class_results.append(deepARV_ensemble.each_class_results)

  # Format and print macro results for each ensemble.
  result_df = print_final_result(cv5_each_class_results)

  return result_df, ensemble_dict

result_df, ensemble_dict =  DeepARV_5_fold_cross_validation()
print('\n ----  Results for each ensemble ---- \n ')
result_df

Loading data...
Distribution of DDI class in training set
  Clinical_label Numeric_label Amount
0   Green         0             17264
1  Yellow         1              1840
2   Amber         2              3816
3     Red         3              1119

########################################

Spliting data into 5 stratified folds...
--------------------------
  Clinical_label Numeric_label Amount
0   Green         0             3453 
1  Yellow         1              368 
2   Amber         2              764 
3     Red         3              224 
--------------------------
  Clinical_label Numeric_label Amount
0   Green         0             3453 
1  Yellow         1              368 
2   Amber         2              763 
3     Red         3              224 
--------------------------
  Clinical_label Numeric_label Amount
0   Green         0             3453 
1  Yellow         1              368 
2   Amber         2              763 
3     Red         3              224 
-----------------

{0:         Accuracy  Precision  Sensitivity  Specificity  F1_score  Balanced Accuracy
 Green  0.7943    0.9387     0.7634       0.8732       0.8420    0.8183            
 Yellow 0.8877    0.3761     0.7092       0.9025       0.4915    0.8059            
 Amber  0.8557    0.5409     0.6060       0.9028       0.5716    0.7544            
 Red    0.9453    0.4568     0.9196       0.9466       0.6104    0.9331            
 Macro  0.8708    0.5781     0.7496       0.9063       0.6289    0.8279            ,
 1:         Accuracy  Precision  Sensitivity  Specificity  F1_score  Balanced Accuracy
 Green  0.7970    0.9284     0.7773       0.8472       0.8462    0.8123            
 Yellow 0.9079    0.4288     0.6141       0.9322       0.5050    0.7732            
 Amber  0.8297    0.4723     0.6265       0.8680       0.5386    0.7472            
 Red    0.9605    0.5450     0.9196       0.9625       0.6844    0.9411            
 Macro  0.8738    0.5936     0.7344       0.9025       0.6435    0.81